### Solves the 1D Advection equation with periodic boundary conditions

In [1]:
import numpy as np
from core import *

In [2]:
import matplotlib.pyplot as plt
from matplotlib import animation, rcParams
from IPython.display import HTML
%matplotlib inline

In [3]:
rcParams['animation.html'] = 'html5'

In [4]:
def animate_hyperbolic(solution):
    fig = plt.figure(figsize=(8,6))
    ax = plt.gca()  
    ax.set_xlim((0, 1))
    ax.set_ylim((-1.5, 1.5))
    ax.set_xlabel('x')
    ax.set_ylabel('u(x)')
    plt.grid(True)
    exact_line, = ax.plot([], [], lw=2)
    approx_line, = ax.plot([], [], '-o')
    plt.close()


    def animate(i):
        exact_line.set_data(grid, solution[:, i])
        approx_line.set_data(grid, np.sin(2 * np.pi * (grid - a*(i+1)*dt)))
        ax.set_title('t = {:01.2f}s'.format(i*dt))
        return exact_line, approx_line,
    
    return animation.FuncAnimation(fig, animate, frames=solution.shape[1]-1, interval=50, blit=True)

In [5]:
a = 1  # Velocity

# Domain's limits
west = 0
east = 1

k = 2  # Operator's order of accuracy
m = 50  # Number of cells
dx = (east - west) / m

t = 1  # Simulation time
dt = dx / abs((k / 2) * a)  # CFL condition for explicit schemes

# 1D Mimetic divergence operator
D = div(k, m, dx)
# 1D 2nd order interpolator
I = interpol(m, 0.5)

# 1D Staggered grid
grid = np.append(np.insert(np.arange(west+dx/2, east, dx), 0, west), east)

# IC
U = np.sin(2 * np.pi * grid)

# Periodic BC imposed on the divergence operator
D[0, 1] = 1. / (2. * dx)
D[0, -2] = -1. / (2. * dx)
D[-1, 1] = 1. / (2. * dx)
D[-1, -2] = -1. / (2. * dx)

# Premultiply out of the time loop (since it doesn't change)
D = -a * dt * 2. * D * I

/Users/nutrik/Documents/Science/scodes/Python/mole/core/div.py:55: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  D[i, j:j+k] = coeffs
/Users/nutrik/Documents/Science/scodes/Python/mole/core/interpol.py:27: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  I[0, 0] = 1.
/Users/nutrik/Documents/Science/scodes/Python/mole/core/interpol.py:28: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  I[-1, -1] = 1.
/Users/nutrik/Documents/Science/scodes/Python/mole/core/interpol.py:37: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  I[i, j:j+2] = avg
/Users/nutrik/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matri

In [6]:
Nt = int(t/dt)
solution = np.zeros((U.shape[0], Nt))
solution[:, 0] = U

# Compute one step using Euler's method
U2 = U + D/2. * U

# Time integration loop
for i in range(Nt):
    solution[:, i] = U2.copy()
    # Leapfrog scheme
    U3 = U + D * U2
    U = U2.copy()
    U2 = U3.copy()

In [7]:
anim = animate_hyperbolic(solution)

In [8]:
anim